In [1]:
import requests
import json

import pandas as pd
from pandas.tseries.frequencies import to_offset

import numpy as np

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import seaborn as sns
import math

from sklearn.preprocessing import MinMaxScaler

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.callbacks import EarlyStopping

import time
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, explained_variance_score, r2_score 
from sklearn.metrics import mean_poisson_deviance, mean_gamma_deviance, accuracy_score
from sklearn.metrics import mean_absolute_percentage_error

from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
import warnings
warnings.filterwarnings('ignore')

In [2]:
segment_size = 100

# 1727 rows
data = pd.read_csv('../Dataset/STB.csv')

# Calculate the number of segments in the dataset
print("len(data)", len(data))
# num_segments = int(len(data)/segment_size)
# # Đọc vào tập dữ liệu bị phân mảnh ngang
# data_chunks = []
# for i in range(num_segments):
#     chunk = pd.read_csv(f'../Dataset/STB.csv')
#     data_chunks.append(chunk)
# data = pd.concat(data_chunks, ignore_index=True)
# data['date'] = pd.to_datetime(data.trunc_time)
# # Áp dụng moving average với cửa sổ 5 và lưu vào cột 'ma_5'
# data['ma_5'] = data['close_price'].rolling(window=5,min_periods=1).mean()
data['ma_5'] = data['close_price'].ewm(span=3, adjust=False, min_periods=0).mean()

# Xóa bỏ các dòng có giá trị null do moving average
data.dropna(inplace=True)
# Chia tập dữ liệu thành tập huấn luyện và tập kiểm tra
# train_data = data.sample(frac=0.8, random_state=42)
# test_data = data.drop(train_data.index)

len(data) 1727


In [3]:
closedf = data[['trunc_time','close_price','ma_5']]
print(closedf)
print("Shape of close dataframe:", closedf.shape)

      trunc_time  close_price          ma_5
0     2016-01-04        12600  12600.000000
1     2016-01-05        12300  12450.000000
2     2016-01-06        12800  12625.000000
3     2016-01-07        12600  12612.500000
4     2016-01-08        12600  12606.250000
...          ...          ...           ...
1722  2022-11-21        16700  16748.526427
1723  2022-11-22        16900  16824.263213
1724  2022-11-23        17500  17162.131607
1725  2022-11-24        18200  17681.065803
1726  2022-11-25        18900  18290.532902

[1727 rows x 3 columns]
Shape of close dataframe: (1727, 3)


In [4]:
import copy

close_stock = closedf.copy()
del closedf['trunc_time']

X_data = copy.deepcopy(closedf)
Y_data = copy.deepcopy(closedf)

del X_data['close_price']
del Y_data['ma_5']

scaler=MinMaxScaler(feature_range=(0,1))

X_data=scaler.fit_transform(np.array(X_data).reshape(-1,1))
Y_data=scaler.fit_transform(np.array(Y_data).reshape(-1,1))
print(X_data, X_data.shape)
print(Y_data, Y_data.shape)

[[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.34368504]
 [0.36209637]
 [0.38371974]] (1727, 1)
[[0.18563923]
 [0.17513135]
 [0.19264448]
 ...
 [0.35726795]
 [0.38178634]
 [0.40630473]] (1727, 1)


In [5]:
del closedf['close_price']
closedf=scaler.fit_transform(np.array(closedf).reshape(-1,1))

print(closedf.shape)

(1727, 1)


In [6]:
# training_size=int(len(closedf)*0.8)
# test_size=len(closedf)-training_size
# train_data,test_data=closedf[0:training_size,:],closedf[training_size:len(closedf),:1]
# print("train_data: ", train_data.shape)
# print("test_data: ", test_data.shape)

In [7]:
# convert an array of values into a dataset matrix
# def create_dataset(dataset, time_step=1):
#     dataX, dataY = [], []
#     for i in range(len(dataset)-time_step-1):
#         a = dataset[i:(i+time_step), 0]   ###i=0, 0,1,2,3-----99   100 
#         dataX.append(a)
#         dataY.append(dataset[i + time_step, 0])
#     return np.array(dataX), np.array(dataY)
def create_dataset(data_set):
    training_size=int(len(data_set)*0.8)
    # test_size=len(data_set)-training_size
    train_data,test_data=data_set[0:training_size,:],data_set[training_size:len(data_set),:1]
    print("train_data: ", train_data)
    print("test_data: ", test_data)
    return train_data,test_data

In [8]:
# reshape into X=t,t+1,t+2,t+3 and Y=t+4
time_step = 10
# print(type(X_data))
X_train, X_test = create_dataset(X_data)
y_train, y_test = create_dataset(Y_data)

print("X_train: ", X_train)
print("y_train: ", y_train.shape)
print("X_test: ", X_test.shape)
print("y_test", y_test.shape)
# print("type(X_test)",type(X_test))

train_data:  [[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.82183932]
 [0.79453479]
 [0.76846483]]
test_data:  [[0.76075172]
 [0.73383373]
 [0.74521013]
 [0.74468948]
 [0.72757657]
 [0.72966385]
 [0.7262726 ]
 [0.72723791]
 [0.74013826]
 [0.74658844]
 [0.75690935]
 [0.7585219 ]
 [0.766424  ]
 [0.78101878]
 [0.77767244]
 [0.78486905]
 [0.80354599]
 [0.81998028]
 [0.81489276]
 [0.81944482]
 [0.81994689]
 [0.81665002]
 [0.80080993]
 [0.79466384]
 [0.80223453]
 [0.80335895]
 [0.80037324]
 [0.79888038]
 [0.76886367]
 [0.73079388]
 [0.71619387]
 [0.71066783]
 [0.70346991]
 [0.70341887]
 [0.71403709]
 [0.70958944]
 [0.70470467]
 [0.70847114]
 [0.71656322]
 [0.70996553]
 [0.70755365]
 [0.70368678]
 [0.69643148]
 [0.68836893]
 [0.68345068]
 [0.68365249]
 [0.68464038]
 [0.69045619]
 [0.68804222]
 [0.68506128]
 [0.68179686]
 [0.68282558]
 [0.67890505]
 [0.67783176]
 [0.66399044]
 [0.65529583]
 [0.63320895]
 [0.61506969]
 [0.63172243]
 [0.63561391]
 [0.63755965]
 [0.64208043]
 [0.65853247]
 [0

In [9]:
# regressor = RandomForestRegressor(n_estimators=100, random_state=0)
# regressor.fit(X_train, y_train)
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import optuna

# Define the objective function for Optuna
def objective(trial):
    n_estimators = trial.suggest_int('n_estimators', 10, 1000)
    max_depth = trial.suggest_int('max_depth', 2, 32)
    min_samples_split = trial.suggest_int('min_samples_split', 2, 10)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 10)
    max_features = trial.suggest_categorical('max_features', ['auto', 'sqrt', 'log2'])
    bootstrap = trial.suggest_categorical('bootstrap', [True, False])
    criterion = trial.suggest_categorical('criterion', ['poisson', 'squared_error', 'absolute_error', 'friedman_mse'])
    random_state = 42

    model = RandomForestRegressor(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        max_features=max_features,
        bootstrap=bootstrap,
        criterion=criterion,
        random_state=random_state
    )

    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    return mse

# Use Optuna to optimize hyperparameters
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)
best_params = study.best_params
print("Optuna: ", best_params)

regressor = RandomForestRegressor(**best_params)
regressor.fit(X_train, y_train)

[I 2023-04-25 14:38:25,558] A new study created in memory with name: no-name-ff3a4c2e-fa1d-4c8d-8cfa-291a41ae6d0c
[I 2023-04-25 14:38:40,032] Trial 0 finished with value: 0.0023792266418214236 and parameters: {'n_estimators': 658, 'max_depth': 28, 'min_samples_split': 2, 'min_samples_leaf': 9, 'max_features': 'auto', 'bootstrap': False, 'criterion': 'absolute_error'}. Best is trial 0 with value: 0.0023792266418214236.
[I 2023-04-25 14:38:45,244] Trial 1 finished with value: 0.001459255147994219 and parameters: {'n_estimators': 526, 'max_depth': 13, 'min_samples_split': 9, 'min_samples_leaf': 9, 'max_features': 'log2', 'bootstrap': True, 'criterion': 'absolute_error'}. Best is trial 1 with value: 0.001459255147994219.
[I 2023-04-25 14:39:03,738] Trial 2 finished with value: 0.001111417811153954 and parameters: {'n_estimators': 765, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 2, 'max_features': 'auto', 'bootstrap': False, 'criterion': 'absolute_error'}. Best is trial 2 w

Optuna:  {'n_estimators': 141, 'max_depth': 23, 'min_samples_split': 7, 'min_samples_leaf': 2, 'max_features': 'sqrt', 'bootstrap': True, 'criterion': 'poisson'}


RandomForestRegressor(criterion='poisson', max_depth=23, max_features='sqrt',
                      min_samples_leaf=2, min_samples_split=7,
                      n_estimators=141)

In [10]:
# Lets Do the prediction 

RF_train_predict=regressor.predict(X_train)
RF_test_predict=regressor.predict(X_test)
# print("Train data prediction:", train_predict)
# # print("Test data prediction:", test_predict)
RF_train_predict = RF_train_predict.reshape(-1,1)
RF_test_predict = RF_test_predict.reshape(-1,1)

print("Train data prediction:", RF_train_predict.shape)
print("Test data prediction:", RF_test_predict.shape)

Train data prediction: (1381, 1)
Test data prediction: (346, 1)


In [11]:
# Transform back to original form

RF_train_predict = scaler.inverse_transform(RF_train_predict)
RF_test_predict = scaler.inverse_transform(RF_test_predict)
RF_original_ytrain = scaler.inverse_transform(y_train.reshape(-1,1)) 
RF_original_ytest = scaler.inverse_transform(y_test.reshape(-1,1)) 



In [12]:
# Evaluation metrices RMSE and MAE
RF_RMSE_train = math.sqrt(mean_squared_error(RF_original_ytrain,RF_train_predict))
RF_MSE_train = mean_squared_error(RF_original_ytrain,RF_train_predict)
RF_MAE_train = mean_absolute_error(RF_original_ytrain,RF_train_predict)

RF_RMSE_test = math.sqrt(mean_squared_error(RF_original_ytest,RF_test_predict))
RF_MSE_test = mean_squared_error(RF_original_ytest,RF_test_predict)
RF_MAE_test = mean_absolute_error(RF_original_ytest,RF_test_predict)

print("Train data RMSE: ", RF_RMSE_train)
print("Train data MSE: ", RF_MSE_train)
print("Test data MAE: ", RF_MAE_train)
print("-------------------------------------------------------------------------------------")
print("Test data RMSE: ", RF_RMSE_test)
print("Test data MSE: ", RF_MSE_test)
print("Test data MAE: ", RF_MAE_test)

Train data RMSE:  149.67289576603935
Train data MSE:  22401.975726991677
Test data MAE:  96.52459726776026
-------------------------------------------------------------------------------------
Test data RMSE:  793.9630415335654
Test data MSE:  630377.3113212301
Test data MAE:  586.5515758378343


In [13]:
RF_EV_train = explained_variance_score(RF_original_ytrain, RF_train_predict)
RF_EV_test = explained_variance_score(RF_original_ytest, RF_test_predict)

print("Train data explained variance regression score:", RF_EV_train)
print("Test data explained variance regression score:", RF_EV_test)

Train data explained variance regression score: 0.9986447999817734
Test data explained variance regression score: 0.9759931802526081


In [14]:
RF_r2_train = r2_score(RF_original_ytrain, RF_train_predict)
RF_r2_test = r2_score(RF_original_ytest, RF_test_predict)

print("Train data R2 score:", RF_r2_train)
print("Test data R2 score:", RF_r2_test)

Train data R2 score: 0.9986447964720665
Test data R2 score: 0.9752072348151101


In [15]:
RF_MGD_train = mean_gamma_deviance(RF_original_ytrain, RF_train_predict)
RF_MGD_test = mean_gamma_deviance(RF_original_ytest, RF_test_predict)
RF_MPD_train = mean_poisson_deviance(RF_original_ytrain, RF_train_predict)
RF_MPD_test = mean_poisson_deviance(RF_original_ytest, RF_test_predict)
print("Train data MGD: ", RF_MGD_train)
print("Test data MGD: ", RF_MGD_test)
print("----------------------------------------------------------------------")
print("Train data MPD: ", RF_MPD_train)
print("Test data MPD: ",RF_MPD_test)

Train data MGD:  0.00010368701903847322
Test data MGD:  0.0008119375806277603
----------------------------------------------------------------------
Train data MPD:  1.4198952714650013
Test data MPD:  22.15785794424433


In [16]:
print(closedf)
# del closedf['close_price']

[[0.18182456]
 [0.17650269]
 [0.18271154]
 ...
 [0.34368504]
 [0.36209637]
 [0.38371974]]


In [17]:
# shift train predictions for plotting
from itertools import cycle
import plotly.express as px
print(RF_train_predict.shape)
print(RF_test_predict.shape)
print(len(closedf))
look_back=time_step

trainPredictPlot = np.empty_like(closedf)
trainPredictPlot[:, :] = np.nan
# trainPredictPlot[look_back:len(RF_train_predict)+look_back, :] = RF_train_predict
trainPredictPlot[0:len(RF_train_predict), :] = RF_train_predict

print("Train predicted data: ", trainPredictPlot)

# shift test predictions for plotting
testPredictPlot = np.empty_like(closedf)
testPredictPlot[:, :] = np.nan
# testPredictPlot[len(RF_train_predict)+(look_back*2)+1:len(closedf)-1, :] = RF_test_predict
testPredictPlot[len(RF_train_predict):len(closedf), :] = RF_test_predict

print("Test predicted data: ", testPredictPlot)

names = cycle(['Original close price','Train predicted close price','Test predicted close price'])


plotdf = pd.DataFrame({'date': close_stock['trunc_time'],
                       'original_close': close_stock['ma_5'],
                      'train_predicted_close': trainPredictPlot.reshape(1,-1)[0].tolist(),
                      'test_predicted_close': testPredictPlot.reshape(1,-1)[0].tolist()})

fig = px.line(plotdf,x=plotdf['date'], y=[plotdf['original_close'],plotdf['train_predicted_close'],
                                          plotdf['test_predicted_close']],
              labels={'value':'Stock price','date': 'Date'})
fig.update_layout(title_text='Comparision between original close price vs predicted close price',
                  plot_bgcolor='white', font_size=15, font_color='black', legend_title_text='Close Price')
fig.for_each_trace(lambda t:  t.update(name = next(names)))

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.show()

(1381, 1)
(346, 1)
1727
Train predicted data:  [[12716.99791608]
 [12482.42732891]
 [12853.13334984]
 ...
 [           nan]
 [           nan]
 [           nan]]
Test predicted data:  [[           nan]
 [           nan]
 [           nan]
 ...
 [17362.64310655]
 [17781.92014001]
 [18550.83547288]]
